### extract ROI from preprocessed .nii.gz

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import nilearn as nl
from nilearn import image, plotting, input_data

In [2]:
def load_sub4d(sub_n,task='videos',max_len=4000,numpy_output=False):
  """ 
  task = videos / recall2 
  """
  ses = 2
  fpath = 'data/fmri/func/sub-1%i_ses-0%i_task-%s_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz'%(sub_n,ses,task)
  try:
    img = nl.image.load_img(fpath)
  except:
    print('NOT FOUND:',fpath)
    return None
  img = img.slicer[:,:,:,:max_len] # lengths differ
  if numpy_output:
    img = img.get_fdata() # nilearn into np
  return img

In [3]:
SUB_NS_FULL = [
  3,4,5,6,7,8,9,10,
  12,13,14,15,17,18,19,
  22,23,24,25,26,27,28,29,
  30,31,32,33,34,35,36,38,
  40,41,42,43,44
  ]

SUB_NS_LOCAL = [
  30,31,32,33,34,35,36,38,
  ]

ROI_NAME_L = [
  'rglasser_AT_net',
  'rglasser_MP_net',
  'rglasser_MTN_net',
  'rglasser_PM_net',
  'rhippocampusAAL',
  # 'rsherlockAvg_fc_thr5_mpfc',
  # 'rsherlockAvg_fc_thr5_pmc',
  # 'rsherlockAvg_fc_thr5_lTPJ',
  # 'rsherlockAvg_fc_thr5_rTPJ',
  # 'rsherlockAvg_fc_thr5_lSFG',
  # 'rsherlockAvg_fc_thr5_rSFG',
  # 'rhippocampusL_AAL',
  # 'rhippocampusR_AAL'  
  ]

In [ ]:
for sub_n in SUB_NS_FULL:
  # load subj
  for task in ['videos','recall2']:
    sub4d = load_sub4d(sub_n,task=task,max_len=4000,numpy_output=False)
    if sub4d == None:
      continue
    for roi_name in ROI_NAME_L:
      print('subj%i'%sub_n,'roi=',roi_name)
      # load & threshold mask
      roi_img = nl.image.threshold_img(
        nl.image.load_img("data/fmri/rois/%s.nii"%roi_name
          ),0.5)
      # plt masks
      plt.figure(figsize=(3,8))
      nl.plotting.plot_glass_brain(roi_img)
      plt.savefig('figures/masks/%s'%roi_name)
      plt.close('all')
      # init & apply mask
      nifti_masker = nl.input_data.NiftiMasker(mask_img=roi_img,high_pass=1/128,t_r=1.5)
      sub4d_masked = nifti_masker.fit_transform(sub4d)
      # save
      save_fpath = "sub-%i_task-%s_roi-%s"%(sub_n,task,roi_name)
      np.save('data/fmri/masked/%s'%save_fpath,sub4d_masked)
  



NOT FOUND: data/fmri/func/sub-13_ses-02_task-videos_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz
NOT FOUND: data/fmri/func/sub-13_ses-02_task-recall2_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz
NOT FOUND: data/fmri/func/sub-14_ses-02_task-videos_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz
NOT FOUND: data/fmri/func/sub-14_ses-02_task-recall2_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz
NOT FOUND: data/fmri/func/sub-15_ses-02_task-videos_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz
NOT FOUND: data/fmri/func/sub-15_ses-02_task-recall2_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz
NOT FOUND: data/fmri/func/sub-16_ses-02_task-videos_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz
NOT FOUND: data/fmri/func/sub-16_ses-02_task-recall2_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz
NOT FOUND: data/fmri/func/sub-17_ses-02_task-videos_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz
NOT FOUND: data/fmri/func/sub-17_ses-02_task-recall2_space-MNI152NLin